#   <span style="color:#5F9EA0">Progetto Python Alessandro Pontini Matricola: 852793</span>

***

In [1]:
# importo librerie
import pandas as pd
import datetime
import numpy as np

In [2]:
# scrivo directory
dire = '/Users/alessandropontini/Documents/db/progetto_python/additional-kiva-snapshot/loans_lenders.csv'
dire1 = '/Users/alessandropontini/Documents/db/progetto_python/additional-kiva-snapshot/loans.csv'
dire2 = '/Users/alessandropontini/Documents/db/progetto_python/additional-kiva-snapshot/lenders.csv'
dire3 = '/Users/alessandropontini/Documents/db/progetto_python/additional-kiva-snapshot/country_stats.csv'

In [3]:
# df temporanei

temp_df3 = pd.DataFrame()
temp_df4 = pd.DataFrame()
temp_df_NA_lenders = pd.DataFrame()
total_lenders_amount = pd.DataFrame()


***

##  <span style="color:red">1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.</span>

***

In [4]:
# leggo il file csv
df_lenders_not_normalized = pd.read_csv(dire)

In [5]:
df_lenders_not_normalized.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [6]:
# impongo index=loan_id per poter usare explode che per ogni loan_id mi prende ogni elemento della lista creata con la strip 
# e mi crea un nuove righe in un nuovo data set
df_lenders_normalized = df_lenders_not_normalized.set_index('loan_id')['lenders'].str.split(pat=',').explode().reset_index()

In [7]:
# mostro risultato
df_lenders_normalized.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


***

##  <span style="color:red">2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.</span>

***

In [8]:
# leggo il file loans, ma per la quantità di dati che devo leggere
# preferisco leggere le date come tali e non come stringhe.
df_loans = pd.read_csv(dire1, parse_dates=['disburse_time', 'planned_expiration_time'])

In [9]:
# ci sono delle date che non hanno disburse_time > planned_expiration_time, non ha senso un prestito che si estingue
# prima della data di sborso, perciò controllo se ce ne sono e in effetti si ci sono.
# creo un df che li contiene df_temp
temp_df = df_loans.loc[df_loans['disburse_time'] > df_loans['planned_expiration_time']]

In [10]:
x = temp_df.loc[:, ('planned_expiration_time')].copy()
temp_df.loc[:, ('disburse_time')] = temp_df.planned_expiration_time
temp_df.loc[:, ('planned_expiration_time')] = x

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
# uso funzione isin per vedere i loan_id che coincidano e cambio df_loan con i nuovi di df_temp corretti
df_loans.loc[df_loans.loan_id.isin(temp_df.loan_id), ['disburse_time', 'planned_expiration_time']] = temp_df[['disburse_time', 'planned_expiration_time']]

In [12]:
# creo una colonna per la duration e faccio differenza, dopo di che mostro il risultato
df_loans['duration'] = df_loans['planned_expiration_time'] - df_loans['disburse_time']
df_loans[['loan_id','duration']]

,loan_id,duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02
...,...,...
1419602,988180,39 days 17:00:03
1419603,988213,39 days 08:40:07
1419604,989109,51 days 14:20:04
1419605,989143,63 days 00:50:02


***

## <span style="color:red">3) Find the lenders that have funded at least twice.</span>

***

In [13]:
temp_df = df_loans[['loan_id', 'status']]

In [14]:
temp_df = df_lenders_normalized.merge(temp_df, how='left', left_on='loan_id', right_on='loan_id')

In [15]:
# creo lista temporanea in cui salvo tutti i loan_name con il value_counts che mi conta quante volte è presente
# e poi chiedo alla lista solo quelli >1
temp_df = temp_df.loc[temp_df.status == 'funded']
temp_df = temp_df.pivot_table(index=['lenders'], aggfunc='size').sort_values(ascending=False)

In [16]:
temp_df[temp_df>1]

lenders
 gooddogg1           142349
 trolltech4460       139191
 gmct                122156
 nms                  93283
 themissionbeltco     70151
                      ...  
 sebastien5634            2
 chaoyi5871               2
jacqui4717                2
 naromie9799              2
 liu9607                  2
Length: 993782, dtype: int64

***

## <span style="color:red">4) For each country, compute how many loans have involved that country as borrowers.</span>


***

In [17]:
# suppongo che in loans che country_code identifichi le nazionalità delle persone perciò con una groupby posso raggrupparle
# e con una size so il numero presente per ogni nazione
df_loans.groupby('country_code').size()

country_code
AF     2337
AL     3075
AM    13952
AZ    10172
BA      608
      ...  
XK     2178
YE     4206
ZA      633
ZM     1277
ZW     5513
Length: 95, dtype: int64

***

## <span style="color:red"> 5) For each country, compute the overall amount of money borrowed.</span>

***

In [18]:
# raggruppo in base allo stato di appartenenza scegliendo di mostrare la colonna loan_amount e sommo i risultati.
df_loans.groupby('country_code')['loan_amount'].sum()

country_code
AF     1967950.0
AL     4307350.0
AM    22950475.0
AZ    14784625.0
BA      477250.0
         ...    
XK     3083025.0
YE     3444000.0
ZA     1006525.0
ZM     1978975.0
ZW     5851875.0
Name: loan_amount, Length: 95, dtype: float64

***

## <span style="color:red">6) Like the previous point, but expressed as a percentage of the overall amount lent.</span>

***

In [19]:
temp = df_loans.funded_amount.sum()

In [20]:
df_loans.groupby('country_code')['loan_amount'].sum()/temp*100

country_code
AF    0.174126
AL    0.381119
AM    2.030683
AZ    1.308160
BA    0.042228
        ...   
XK    0.272789
YE    0.304729
ZA    0.089058
ZM    0.175102
ZW    0.517780
Name: loan_amount, Length: 95, dtype: float64

***

## <span style="color:red">7) Like the three previous points, but split for each year (with respect to disburse time).</span>

***

In [21]:
temp_df = df_loans.set_index('disburse_time')

In [22]:
# vedo quante persone all'anno
temp_df.groupby(['country_code', pd.Grouper(freq='Y')]).size()

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00     408
              2008-12-31 00:00:00+00:00     370
              2009-12-31 00:00:00+00:00     678
              2010-12-31 00:00:00+00:00     632
              2011-12-31 00:00:00+00:00     247
                                           ... 
ZW            2013-12-31 00:00:00+00:00     426
              2014-12-31 00:00:00+00:00    2078
              2015-12-31 00:00:00+00:00     600
              2016-12-31 00:00:00+00:00     808
              2017-12-31 00:00:00+00:00    1079
Length: 740, dtype: int64

In [23]:
# faccio le somme
temp_df.groupby(['country_code', pd.Grouper(freq='Y')])['loan_amount'].sum()

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00     194975.0
              2008-12-31 00:00:00+00:00     365375.0
              2009-12-31 00:00:00+00:00     585125.0
              2010-12-31 00:00:00+00:00     563350.0
              2011-12-31 00:00:00+00:00     245125.0
                                             ...    
ZW            2013-12-31 00:00:00+00:00     678525.0
              2014-12-31 00:00:00+00:00    1311575.0
              2015-12-31 00:00:00+00:00     723625.0
              2016-12-31 00:00:00+00:00     788600.0
              2017-12-31 00:00:00+00:00    1237600.0
Name: loan_amount, Length: 740, dtype: float64

In [24]:
# eseguo operazioni punto 6 
temp_df2 = df_loans['loan_amount'].sum()

In [25]:
temp_df.groupby(['country_code', pd.Grouper(freq='Y')])['loan_amount'].sum()/temp_df2*100

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00    0.016503
              2008-12-31 00:00:00+00:00    0.030926
              2009-12-31 00:00:00+00:00    0.049527
              2010-12-31 00:00:00+00:00    0.047683
              2011-12-31 00:00:00+00:00    0.020748
                                             ...   
ZW            2013-12-31 00:00:00+00:00    0.057432
              2014-12-31 00:00:00+00:00    0.111015
              2015-12-31 00:00:00+00:00    0.061250
              2016-12-31 00:00:00+00:00    0.066749
              2017-12-31 00:00:00+00:00    0.104754
Name: loan_amount, Length: 740, dtype: float64

***

## <span style="color:red">8) For each lender, compute the overall amount of money lent.</span>

***

In [26]:
# uso un df temporaneo per immagazzinare la join tra df_lenders_normalized e df_loans con solo i campi loan_id
# e loan_amount che usero per i conti successivamente
temp_df = df_lenders_normalized.merge(df_loans[['loan_id', 'loan_amount']], how='left', on='loan_id')

In [27]:
# salvo in un df temp 2 il raggruppamento secondo count, per contare il numero totale di ogni loan id
temp_df2 = temp_df.groupby('loan_id')['lenders'].count()

In [28]:
# faccio ultima merge per avere nel mio data set accanto alla colonna di loan id la colonna dei lenders_y
# che corrisponde al mio count
temp_df = temp_df.merge(temp_df2, how='left', on='loan_id')

In [29]:
temp_df

,loan_id,lenders_x,loan_amount,lenders_y
0,483693,muc888,1225.0,40
1,483693,sam4326,1225.0,40
2,483693,camaran3922,1225.0,40
3,483693,lachheb1865,1225.0,40
4,483693,rebecca3499,1225.0,40
...,...,...,...,...
28293926,1206425,trogdorfamily7622,325.0,8
28293927,1206425,danny6470,325.0,8
28293928,1206425,don6118,325.0,8
28293929,1206486,alan5175,150.0,2


In [30]:
# creo nuova colonna per segnare le singole quote per ogni loan_id
temp_df['single_amount']= temp_df['loan_amount']/temp_df['lenders_y']

In [31]:
# ora raggruppo per ogni lender
temp_df.groupby('lenders_x')['single_amount'].sum()

lenders_x
 000               1672.618411
 00000             1380.693644
 0002              2472.563566
 00mike00            52.631579
 0101craign0101    2623.565117
                      ...     
zzanita              87.500000
zzcyna7269           55.357143
zzinnia              38.000000
zzmcfate           2287.291955
zzrvmf8538          106.417625
Name: single_amount, Length: 1639026, dtype: float64

***

## <span style='color:red'> 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.</span>

***

## Problema di fondo che abbiamo trovato che il data set estremamente scarno con molti NULL, perciò dovevamo distribuire i soldi borrowed amolti valori NULL. Personalmento ho optato per la distribuzione del totale loan_amount dei null secondo percentuali di nazionalità

***

In [32]:
# inizio aggiungendo ai miei dataset anche le informazioni dei lenders_info
df_lenders_info = pd.read_csv(dire2)

In [33]:
# merge in una temporanea di df_lenders con la normalizzata 
# per avere loan_id
temp_df = df_lenders_info[['permanent_name', 'country_code']].merge(df_lenders_normalized, how='left', left_on='permanent_name', right_on='lenders')

In [34]:
# temp_df2 contiene tutte le righe in cui lenders è presente in loan e non ha lenders == null
temp_df2 = temp_df.dropna(subset=['lenders'])

In [35]:
# contiene tutti i borrowed
temp_df3 = temp_df.loc[temp_df.lenders.isna()]

In [36]:
# analizzando il temp df 3 mi accorgo che ho qualche country code ma non ho nessun loan amount
# per tanto decido di non eseguire altri compiti
temp_df3

,permanent_name,country_code,loan_id,lenders
0,qian3013,NaN,NaN,NaN
1,reena6733,NaN,NaN,NaN
2,mai5982,NaN,NaN,NaN
3,andrew86079135,NaN,NaN,NaN
4,nguyen6962,NaN,NaN,NaN
...,...,...,...,...
3449032,janet7309,NaN,NaN,NaN
3449033,pj4198,NaN,NaN,NaN
3449034,maria2141,US,NaN,NaN
3449035,simone9846,NaN,NaN,NaN


In [37]:
temp_df2

,permanent_name,country_code,loan_id,lenders
16,naresh2074,US,1059280.0,naresh2074
17,naresh2074,US,1055468.0,naresh2074
48,jinxian2217,NaN,1292306.0,jinxian2217
49,sreedevi9583,NaN,1052062.0,sreedevi9583
52,nicole98153893,NaN,1137907.0,nicole98153893
...,...,...,...,...
3449014,bonnie9739,US,810311.0,bonnie9739
3449015,bonnie9739,US,846429.0,bonnie9739
3449016,bonnie9739,US,809520.0,bonnie9739
3449018,cody7091,US,441719.0,cody7091


In [38]:
# aggiungiamo anche la colonna loan_amount per avere un totale dei lenders/loan/country ( potrebbero esserci dei NULL)
temp_df2=temp_df2.merge(df_loans[['loan_id', 'loan_amount']],how='left',on='loan_id')

In [39]:
temp_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1387432 entries, 0 to 1387431
Data columns (total 5 columns):
permanent_name    1387432 non-null object
country_code      1097763 non-null object
loan_id           1387432 non-null float64
lenders           1387432 non-null object
loan_amount       1387428 non-null float64
dtypes: float64(2), object(3)
memory usage: 63.5+ MB


In [40]:
# metto i null a 0 di loan amount che sono 4
temp_df2['loan_amount'] = temp_df2['loan_amount'].fillna(0)

In [41]:
# inizio controllando quelli con country valore che esiste, perdo quasi 3000000 righe
df_lenders_loans_country = temp_df2.dropna(subset=['country_code'])

In [42]:
df_lenders_loans_country = df_lenders_loans_country.groupby('country_code')['loan_amount'].sum()

In [43]:
total_lent = df_lenders_loans_country.sum()

In [44]:
df_lenders_loans_country = df_lenders_loans_country.reset_index()

In [45]:
# cosi facendo ho percentuale del totale per ogni country che è il punto che mi serve
df_lenders_loans_country['percentage'] = round(df_lenders_loans_country['loan_amount']/total_lent,10)

In [46]:
df_lenders_loans_country

,country_code,loan_amount,percentage
0,AD,1200.0,0.000001
1,AE,1760225.0,0.001930
2,AF,68075.0,0.000075
3,AL,16650.0,0.000018
4,AM,23575.0,0.000026
...,...,...,...
190,WF,2100.0,0.000002
191,XK,3750.0,0.000004
192,YE,15175.0,0.000017
193,ZA,509075.0,0.000558


In [47]:
temp_df2 = temp_df2.loc[temp_df2.country_code.isna()]

In [48]:
temp_df2

,permanent_name,country_code,loan_id,lenders,loan_amount
2,jinxian2217,NaN,1292306.0,jinxian2217,1400.0
3,sreedevi9583,NaN,1052062.0,sreedevi9583,650.0
4,nicole98153893,NaN,1137907.0,nicole98153893,200.0
6,haitang3172,NaN,1291625.0,haitang3172,300.0
7,jishi5401,NaN,1060071.0,jishi5401,400.0
...,...,...,...,...,...
1387388,tricia9236,NaN,699175.0,tricia9236,1450.0
1387396,marwan8219,NaN,735870.0,marwan8219,475.0
1387399,eric9768,NaN,898756.0,eric9768,225.0
1387405,nicole1492,NaN,485083.0,nicole1492,500.0


In [49]:
total_amount = temp_df2.loan_amount.sum()

In [50]:
total_amount

251447675.0

In [51]:
# li moltiplico per la percentuale dei country che conosco lenders e poi li sommo con i precedenti
x = total_amount * df_lenders_loans_country.percentage

df_lenders_loans_country['total_lenders_amount_with_no_country'] = x

In [52]:
df_lenders_loans_country['total_lent'] = df_lenders_loans_country.loan_amount + df_lenders_loans_country.total_lenders_amount_with_no_country

In [53]:
df_lenders_loans_country

,country_code,loan_amount,percentage,total_lenders_amount_with_no_country,total_lent
0,AD,1200.0,0.000001,330.854851,1.530855e+03
1,AE,1760225.0,0.001930,485319.006649,2.245544e+06
2,AF,68075.0,0.000075,18769.236266,8.684424e+04
3,AL,16650.0,0.000018,4590.629913,2.124063e+04
4,AM,23575.0,0.000026,6499.972688,3.007497e+04
...,...,...,...,...,...
190,WF,2100.0,0.000002,579.008561,2.679009e+03
191,XK,3750.0,0.000004,1033.927695,4.783928e+03
192,YE,15175.0,0.000017,4183.963588,1.935896e+04
193,ZA,509075.0,0.000558,140359.198555,6.494342e+05


In [54]:
# no duplicates
a = df_loans[df_loans.duplicated()]
print(a)

Empty DataFrame
Columns: [loan_id, loan_name, original_language, description, description_translated, funded_amount, loan_amount, status, activity_name, sector_name, loan_use, country_code, country_name, town_name, currency_policy, currency_exchange_coverage_rate, currency, partner_id, posted_time, planned_expiration_time, disburse_time, raised_time, lender_term, num_lenders_total, num_journal_entries, num_bulk_entries, tags, borrower_genders, borrower_pictured, repayment_interval, distribution_model, duration]
Index: []

[0 rows x 32 columns]


In [55]:
temp_df = df_loans.groupby('country_code')['loan_amount'].sum()

In [56]:
temp_df = temp_df.reset_index()

In [57]:
del df_lenders_loans_country['total_lenders_amount_with_no_country']

In [58]:
df_lenders_loans_country = df_lenders_loans_country.merge(temp_df,how='left', on='country_code')

In [59]:
df_lenders_loans_country.loan_amount_y = df_lenders_loans_country.loan_amount_y.fillna(0)

In [60]:
df_lenders_loans_country['lent_borrowed'] = df_lenders_loans_country.total_lent - df_lenders_loans_country.loan_amount_y

In [61]:
df_lenders_loans_country

,country_code,loan_amount_x,percentage,total_lent,loan_amount_y,lent_borrowed
0,AD,1200.0,0.000001,1.530855e+03,0.0,1.530855e+03
1,AE,1760225.0,0.001930,2.245544e+06,0.0,2.245544e+06
2,AF,68075.0,0.000075,8.684424e+04,1967950.0,-1.881106e+06
3,AL,16650.0,0.000018,2.124063e+04,4307350.0,-4.286109e+06
4,AM,23575.0,0.000026,3.007497e+04,22950475.0,-2.292040e+07
...,...,...,...,...,...,...
190,WF,2100.0,0.000002,2.679009e+03,0.0,2.679009e+03
191,XK,3750.0,0.000004,4.783928e+03,3083025.0,-3.078241e+06
192,YE,15175.0,0.000017,1.935896e+04,3444000.0,-3.424641e+06
193,ZA,509075.0,0.000558,6.494342e+05,1006525.0,-3.570908e+05


***

## <span style='color:red'>10) Which country has the highest ratio between the difference computed at the previous point and the population?</span>

***

In [62]:
# importo data set che necessito
df_country_stat = pd.read_csv(dire3)

In [63]:
temp_df = df_country_stat.groupby('country_code')['population'].sum()

In [64]:
temp_df = temp_df.reset_index()

In [65]:
df_lenders_loans_country = df_lenders_loans_country.merge(temp_df,how='left', on='country_code')

In [66]:
# seh ho popolazione nan metto il primo quantile
first_quantile = df_lenders_loans_country.population.quantile(0.25)
df_lenders_loans_country.population = df_lenders_loans_country.population.fillna(first_quantile)

In [67]:
df_lenders_loans_country

,country_code,loan_amount_x,percentage,total_lent,loan_amount_y,lent_borrowed,population
0,AD,1200.0,0.000001,1.530855e+03,0.0,1.530855e+03,4301080.25
1,AE,1760225.0,0.001930,2.245544e+06,0.0,2.245544e+06,9400145.00
2,AF,68075.0,0.000075,8.684424e+04,1967950.0,-1.881106e+06,35530081.00
3,AL,16650.0,0.000018,2.124063e+04,4307350.0,-4.286109e+06,2930187.00
4,AM,23575.0,0.000026,3.007497e+04,22950475.0,-2.292040e+07,2930450.00
...,...,...,...,...,...,...,...
190,WF,2100.0,0.000002,2.679009e+03,0.0,2.679009e+03,4301080.25
191,XK,3750.0,0.000004,4.783928e+03,3083025.0,-3.078241e+06,1895250.00
192,YE,15175.0,0.000017,1.935896e+04,3444000.0,-3.424641e+06,28250420.00
193,ZA,509075.0,0.000558,6.494342e+05,1006525.0,-3.570908e+05,56717156.00


In [68]:
#calcolo percentuale su i lent-borrowed e la popolazione, successivamente trovo l'idmax per la percentuale che mi dice essere CANADA
df_lenders_loans_country['percentage'] = round(df_lenders_loans_country['lent_borrowed']/df_lenders_loans_country['population']*100, 2)
#temp_df3=temp_df3.reset_index()

In [69]:
max_ratio = df_lenders_loans_country.loc[df_lenders_loans_country['percentage'].idxmax()]

In [70]:
max_ratio

country_code              NO
loan_amount_x    1.31226e+07
percentage            315.54
total_lent       1.67407e+07
loan_amount_y              0
lent_borrowed    1.67407e+07
population       5.30538e+06
Name: 132, dtype: object

***

## <span style='color:red'>11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?</span>

***

In [71]:
temp_df2 = df_country_stat.groupby('country_code')[['country_code','population','population_below_poverty_line']].head()

In [72]:
temp_df2['population_below_poverty_line'] = temp_df2['population_below_poverty_line']/100

In [74]:
mean = temp_df2.population_below_poverty_line.mean()
temp_df2.population_below_poverty_line = temp_df2.population_below_poverty_line.fillna(mean)

In [75]:
temp_df2

,country_code,population,population_below_poverty_line
0,IN,1339180127,0.21900
1,NG,190886311,0.70000
2,MX,129163276,0.46200
3,PK,197015955,0.29500
4,BD,164669751,0.31500
...,...,...,...
169,SO,14742523,0.28477
170,CF,4659080,0.28477
171,WS,196440,0.28477
172,PS,4920724,0.28477


In [77]:
temp_df2['population_below_poverty_line'] = temp_df2['population'] - (temp_df2['population_below_poverty_line']*temp_df2['population'])

In [83]:
temp_df2 = temp_df2.merge(df_lenders_loans_country[['country_code', 'lent_borrowed']], left_on='country_code', right_on='country_code')

In [85]:
mean = temp_df2.lent_borrowed.mean()
temp_df2.lent_borrowed = temp_df2.lent_borrowed.fillna(mean)

In [86]:
temp_df2['percentage_poverty'] = round(temp_df2['lent_borrowed']/temp_df2['population_below_poverty_line']*100, 2)

In [90]:
max_ratio = temp_df2.loc[temp_df2['percentage_poverty'].idxmax()]

In [91]:
max_ratio

country_code                              NO
population                           5305383
population_below_poverty_line    3.79457e+06
lent_borrowed                    1.67407e+07
percentage_poverty                    441.18
Name: 146, dtype: object

***

## <span style='color:red'>12) For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.</span>

***

### Decido di eliminare tutte le colonne con NaN, in particolar modo tutte le colonne che non presentano disburse_time oppure planned_expiration_time

***

In [150]:
temp_df = df_loans.dropna()

In [151]:
temp_df.loc[:,('disburse_time')]=temp_df.loc[:,('disburse_time')].dt.tz_localize(None)
temp_df.loc[:,('planned_expiration_time')]=temp_df.loc[:,('planned_expiration_time')].dt.tz_localize(None)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [152]:
temp_df['differenza'] = temp_df['planned_expiration_time'].dt.year - temp_df['disburse_time'].dt.year

<ipython-input-152-8c6a6d9a8ff2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['differenza'] = temp_df['planned_expiration_time'].dt.year - temp_df['disburse_time'].dt.year


In [153]:
temp_df = temp_df[['planned_expiration_time', 'disburse_time', 'differenza', 'loan_amount', 'duration']]

In [154]:
# mi accorgo che ho solo 1 o 0 come differenza perciò preparo programma apposta
temp_df.groupby('differenza').size()

differenza
0    415502
1     63335
dtype: int64

In [155]:
# tengo solo i giorni
temp_df.duration = temp_df.duration.dt.days

In [156]:
def func(k):
        x = round(( k.loan_amount * k.duration)/(31+365+30),2)
        y = k['planned_expiration_time'].year
        return [y,x]

def func2(k):
        lim_inf = datetime.datetime(k.planned_expiration_time.year, 1,1)
        z = k.planned_expiration_time - lim_inf
        z = z.days
        x = round(( k.loan_amount * z)/(31+365+30),2) 
        y = k.planned_expiration_time.year
        lim_sup = datetime.datetime(k.disburse_time.year, 12, 31)
        f = lim_sup - k.disburse_time
        f = f.days
        u = round(( k.loan_amount * f)/(31+365+30),2) 
        v = k.disburse_time.year
        return [[y,x],[v,u]]


In [157]:
temp_df1 = temp_df.loc[temp_df.differenza == 0]
listx = []

In [158]:
# applico funzione per gli 0 anni di differenza
listx = temp_df1.apply(lambda x : func(x), axis=1)

In [159]:
d1

{'tot': 13           [2013, 2.82]
 71          [2014, 28.52]
 74          [2016, 71.48]
 75         [2016, 202.46]
 79          [2013, 22.59]
                 ...      
 1419576     [2015, 52.11]
 1419586    [2015, 540.61]
 1419589      [2015, 13.5]
 1419603     [2015, 25.35]
 1419604    [2015, 267.55]
 Name: data2, Length: 63335, dtype: object}

In [160]:
d1 = {'tot' : listx}
df2 = pd.DataFrame(d1)
df3 = df2.tot.apply(pd.Series)
df3.columns = ['year', 'total']

In [172]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542172 entries, 0 to 542171
Data columns (total 2 columns):
year     542172 non-null float64
total    542172 non-null float64
dtypes: float64(2)
memory usage: 8.3 MB


In [163]:
# applico funzione per gli 1 anni 
temp_df1 = temp_df.loc[temp_df.differenza == 1]
listy = []

In [164]:
listy = temp_df1.apply(lambda x : func2(x), axis=1)

In [165]:
# ottengo due df di liste con i dati immagazzinati dal limite superiore e dal limite inferiore
d1 = {'tot' : listy}
df2 = pd.DataFrame(d1)
df4 = df2.tot.apply(pd.Series)
df4.columns = ['data1', 'data2']
df2 = df4['data1']
df4 = df4['data2']

In [166]:
# primo limite
d1 = {'tot' : df2}
df2 = pd.DataFrame(d1)
df5 = df2.tot.apply(pd.Series)
df5.columns = ['year', 'total']

In [170]:
# secondo limite
d1 = {'tot' : df4}
df2 = pd.DataFrame(d1)
df6 = df2.tot.apply(pd.Series)
df6.columns = ['year', 'total']

In [168]:
# appendo entrambi i data set creati al primo creato
df3 = df3.append(df5, ignore_index=True)

In [171]:
df3 = df3.append(df6, ignore_index=True)

In [176]:
df3.year = df3.year.astype('int')

In [177]:
df3.groupby('year').size()

year
2011         1
2012       745
2013     10801
2014     79474
2015    114483
2016    145724
2017    180452
2018     10492
dtype: int64

In [178]:
df3.groupby('year').sum()

,total
year,
2011,29.93
2012,74010.36
2013,1443746.34
2014,8845883.56
2015,10590086.70
2016,11577277.04
2017,13381987.53
2018,214480.37


# TEMPO IMPIEGATO 12 PUNTI : 5.30min 
# COMPUTER UTILIZZATO : MACBOOK PRO 19
# PROCESSORE : 1,4 GHz Intel Core i5 quad-core
# MEMORIA : 8 GB 2133 MHz LPDDR3
# SCHEDA GRAFICA : Intel Iris Plus Graphics 645 1536 MB
